# GOES historical data download with respect to labeled AR data

First, load the labeled_AR data files

In [2]:
import os

# Directory where files are stored
download_dir = 'labeled_AR'

# List to store file names
files = []

# Iterate over files in the directory
for file_name in os.listdir(download_dir):
    if file_name.endswith('.nc'):
        files.append(os.path.join(download_dir, file_name))

# Sort the list of file names
files.sort()
files

['labeled_AR/data-1996-06-09-01-1_0.nc',
 'labeled_AR/data-1996-07-11-01-1_0.nc',
 'labeled_AR/data-1996-07-18-01-1_0.nc',
 'labeled_AR/data-1996-07-18-01-1_1.nc',
 'labeled_AR/data-1996-07-18-01-1_2.nc',
 'labeled_AR/data-1996-09-01-01-1_0.nc',
 'labeled_AR/data-1996-09-01-01-1_1.nc',
 'labeled_AR/data-1996-09-12-01-1_0.nc',
 'labeled_AR/data-1996-09-26-01-1_0.nc',
 'labeled_AR/data-1996-09-26-01-1_1.nc',
 'labeled_AR/data-1996-10-03-01-1_0.nc',
 'labeled_AR/data-1996-10-03-01-1_1.nc',
 'labeled_AR/data-1997-06-01-01-1_0.nc',
 'labeled_AR/data-1997-06-02-01-1_0.nc',
 'labeled_AR/data-1997-06-05-01-1_0.nc',
 'labeled_AR/data-1997-06-18-01-1_0.nc',
 'labeled_AR/data-1997-07-09-01-1_0.nc',
 'labeled_AR/data-1997-07-09-01-1_1.nc',
 'labeled_AR/data-1997-08-14-01-1_0.nc',
 'labeled_AR/data-1997-08-14-01-1_1.nc',
 'labeled_AR/data-1997-08-14-01-1_2.nc',
 'labeled_AR/data-1997-08-14-01-1_3.nc',
 'labeled_AR/data-1997-08-29-01-1_0.nc',
 'labeled_AR/data-1997-10-12-01-1_0.nc',
 'labeled_AR/dat

In [14]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timedelta

# Directory to save the downloaded files
save_dir = ""
os.makedirs(save_dir, exist_ok=True)

# Base URL
base_url = "https://www.ncei.noaa.gov/data/gridsat-goes/access/goes"

# Specify start and end dates
start_date = datetime(1996, 9, 26, 0, 0)
end_date = datetime(1996, 10, 5, 23, 0)

# Function to generate URLs based on date range
def generate_urls(start_date, end_date):
    urls = []
    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime("%Y.%m.%d.%H00")
        filename = f"GridSat-GOES.goes08.{date_str}.v01.nc"
        url = f"{base_url}/{current_date.year}/{current_date.strftime('%m')}/{filename}"
        urls.append(url)
        current_date += timedelta(hours=1)
    return urls

# Function to download a file
def download_file(url):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        filename = os.path.join(save_dir, os.path.basename(url))
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded {filename}")
    else:
        print(f"Failed to download {url}")

# Generate the list of URLs to download
urls = generate_urls(start_date, end_date)

# Use ThreadPoolExecutor to download files concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(download_file, urls)

print("All files downloaded.")


FileNotFoundError: [Errno 2] No such file or directory: ''

In [4]:
files = os.listdir(os.getcwd())
files

['.bash_history',
 '.ipython',
 'shared',
 'NWM_CNN_california_AR_2023',
 'GOES_historical.ipynb',
 '.wget-hsts',
 'data_science_principals',
 '.cache',
 'labeled_AR',
 'reference_CONUS.gpkg',
 'geopandas',
 'climatenet_data',
 'neuralhydrology',
 'wget-log',
 'SI_fellows_2024_introductions',
 '.jupyter',
 'Untitled4.ipynb',
 'GOES_historical',
 'Untitled3.ipynb',
 '.local',
 'Untitled1.ipynb',
 'Untitled2.ipynb',
 'labeled_AR.ipynb',
 '.ipynb_checkpoints',
 '.conda',
 'hydrofabric',
 '.jupyter-server-log.txt',
 '.config',
 'CAMELS_data_sample']

In [15]:
import os

# Directory where files are stored
download_dir = ''

# List to store file names
files = []

# Iterate over files in the directory
for file_name in os.listdir(download_dir):
    if file_name.endswith('.nc'):
        files.append(os.path.join(download_dir, file_name))

# Sort the list of file names
files.sort()
files


FileNotFoundError: [Errno 2] No such file or directory: ''

In [12]:
!pip install cartopy

  Using cached shapely-2.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 444.5 kB/s eta 0:00:00 0:00:01
  Using cached pyproj-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 35.5 MB/s eta 0:00:0000:0100:01
Using cached pyproj-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 1.8 MB/s eta 0:00:00
Using cached shapely-2.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.5 MB)


In [16]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os

# Directory containing the NetCDF files
data_dir = ''
# List of NetCDF files to process
nc_files = [f for f in os.listdir(data_dir) if f.endswith('.nc')]

# Define the latitude and longitude bounds for the US
lat_bounds = [10, 100]  # Southernmost and northernmost points of the continental US
lon_bounds = [-180, -40]  # Westernmost and easternmost points of the continental US

# Function to process and plot data from a single file
def process_and_plot(nc_file):
    # Open the NetCDF file
    dataset = xr.open_dataset(os.path.join(data_dir, nc_file))
    
    # Select the variable you want to plot
    variable = dataset['ch3']
    
    # Crop the data to the US bounds
    variable_us = variable.sel(lat=slice(lat_bounds[0], lat_bounds[1]), lon=slice(lon_bounds[0], lon_bounds[1]))
    
    # Create a figure and axis with a globe projection
    fig = plt.figure(figsize=(10, 6))
    ax = plt.axes(projection=ccrs.Mercator())
    
    # Plot the variable on the globe
    variable_us.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='viridis')  # Added cmap for better visualization
    
    # Add coastlines and gridlines
    ax.coastlines()
    ax.gridlines(draw_labels=True)
    
    # Set the title
    plt.title(f'Variable from {nc_file} (Cropped to US)')
    
    # Show the plot
    plt.show()

# Iterate through the NetCDF files and process them
for nc_file in nc_files:
    process_and_plot(nc_file)


FileNotFoundError: [Errno 2] No such file or directory: ''